## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os


sys.path.append("../")

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.

city_path = os.path.join("..", "Vacation_Search", "WeatherPy_vacation.csv")

vacation_df = pd.read_csv(city_path)
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,15,Kendari,ID,79.43,overcast clouds,-3.9450,122.4989,Hotel Wixel
1,19,Pochutla,MX,85.82,clear sky,15.7432,-96.4661,Hotel Posada San Jose
2,23,Pierre,US,75.99,overcast clouds,44.3683,-100.3510,Ramkota Hotel
3,28,Samana,DO,75.56,overcast clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
4,29,Lazaro Cardenas,MX,78.35,few clouds,17.9583,-102.2000,Hotel Quinta Antigua


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
#Build a US only DF
USonly_df = vacation_df[(vacation_df["Country"] == 'US')]
USonly_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,23,Pierre,US,75.99,overcast clouds,44.3683,-100.3510,Ramkota Hotel
5,33,Makakilo City,US,82.72,broken clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
11,50,Kapaa,US,82.38,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
30,110,Truth Or Consequences,US,80.91,scattered clouds,33.1284,-107.2528,Charles Motel
33,118,Ewa Beach,US,84.15,broken clouds,21.3156,-158.0072,Uncle Junior Robert


In [6]:
#this works
vacation_start = USonly_df.sample()
vacation_end = vacation_start
vacation_stop1 = USonly_df.sample()
vacation_stop2 = USonly_df.sample()
vacation_stop3 = USonly_df.sample()
vacation_stop1


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,33,Makakilo City,US,82.72,broken clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [7]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = USonly_df.loc[USonly_df['City']=='Pierre']
vacation_end = vacation_start
vacation_stop1 = USonly_df.loc[USonly_df['City']=='Fort Dodge']
vacation_stop2 = USonly_df.loc[USonly_df['City']=='Plattsburgh']
vacation_stop3 = USonly_df.loc[USonly_df['City']=='Roma']
#vacation_stop3 = vacation_df.loc[19]


In [8]:

# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(vacation_start[["Lat","Lng"]].to_numpy()[0])
end = tuple(vacation_end[["Lat","Lng"]].to_numpy()[0])
stop1 = tuple(vacation_stop1[["Lat","Lng"]].to_numpy()[0])
stop2 = tuple(vacation_stop2[["Lat","Lng"]].to_numpy()[0])
stop3 = tuple(vacation_stop3[["Lat","Lng"]].to_numpy()[0])



In [9]:


# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
starttoend = gmaps.directions_layer(start, end, waypoints = [stop1,stop2, stop3], travel_mode='DRIVING')
fig.add_layer(starttoend)
fig




Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,23,Pierre,US,75.99,overcast clouds,44.3683,-100.3510,Ramkota Hotel
1,399,Fort Dodge,US,75.45,few clouds,42.4975,-94.1680,"Country Inn & Suites by Radisson, Fort Dodge, IA"
2,694,Plattsburgh,US,77.32,clear sky,44.6995,-73.4529,La Quinta Inn & Suites by Wyndham Plattsburgh
3,597,Roma,US,75.88,clear sky,43.2128,-75.4557,Wingate By Wyndham Rome
4,23,Pierre,US,75.99,overcast clouds,44.3683,-100.3510,Ramkota Hotel


In [11]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))